###  보이스챗봇
1. 사용자 입력을 음성으로 받는다.
2. STT를 적용하여 텍스트로 변환한다.
3. 변환된 텍스트를 입력으로 하여 프롬프트 엔지니어링을 해 api 요청을 보낸다.
4. 반환받은 응답을 TTS를 적용하여 음성으로 재생한다.
(+) 2에서 입력된 텍스트와 4에서 반환된 응답을 채팅 내역 보듯이 (카카오톡 대화처럼) 현출되도록 출력한다.

In [ ]:
!pip install openai python-dotenv gtts SpeechRecognition pydub pyaudio IPython

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import io
import random
from IPython.display import display, HTML, Audio, clear_output
import ipywidgets as widgets
from threading import Thread
import time

In [ ]:

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

print("라이브러리 로드 ")


In [ ]:
# 팩폭해주는 보이스 챗봇 - Jupyter Notebook



# ===== 셀 3: 팩폭 챗봇 클래스 정의 =====

class FactBombChatbot:
    def __init__(self):
        self.recognizer = sr.Recognizer()
        self.chat_history = []
        self.is_listening = False
        
        # 팩폭 프롬프트 템플릿들
        self.factbomb_prompts = {
            '사랑': [
                '사랑은 호르몬의 장난이에요. 도파민, 세로토닌, 옥시토신의 칵테일이죠. 하지만 그래도 아름다운 착각이니까 즐기세요!',
                '진짜 사랑은 상대방의 단점까지 사랑하는 거예요. 지금 그 사람 코골이 소리도 귀엽다고 생각하시나요?'
            ],
            '다이어트': [
                '다이어트의 진실: 칼로리 IN < 칼로리 OUT. 이것만 지키면 됩니다. 기적의 다이어트는 없어요!',
                '운동 안 하고 다이어트약만 먹는다고요? 그건 돈 다이어트밖에 안 돼요! 몸 다이어트는 땀으로 하는 거예요'
            ],
            '성공': [
                '성공의 비밀은 재능 10%, 노력 90%예요. 지금 소파에서 성공 유튜브 보고 있다면... 그건 망상이에요!',
                '성공한 사람들의 공통점? 실패를 두려워하지 않아요. 당신은 실패를 두려워해서 시작도 안 하고 있죠?'
            ],
            '돈': [
                '돈 없다고 하면서 배달음식 시키고, 커피 사 마시죠? 가계부부터 써보세요. 충격받을 거예요!',
                '부자가 되는 법: 쓰는 돈 < 버는 돈. 간단하죠? 근데 왜 안 해요? 자제력 문제예요!'
            ],
            '공부': [
                '공부 안 되는 이유? 핸드폰이 손에 붙어있으니까요! 스마트폰 중독부터 치료하세요',
                '집중력이 없다고요? 좋아하는 게임할 때는 몇 시간도 집중하잖아요. 핑계 그만!'
            ]
        }
        
        print("🤖 팩폭 챗봇이 준비되었습니다!")
    
    def speech_to_text(self):
        """음성을 텍스트로 변환 (STT)"""
        try:
            with sr.Microphone() as source:
                print("🎤 말씀하세요...")
                self.recognizer.adjust_for_ambient_noise(source, duration=1)
                audio = self.recognizer.listen(source, timeout=5, phrase_time_limit=10)
                
            print("🔄 음성을 텍스트로 변환 중...")
            text = self.recognizer.recognize_google(audio, language='ko-KR')
            return text
            
        except sr.UnknownValueError:
            return "음성을 인식할 수 없습니다."
        except sr.RequestError as e:
            return f"음성 인식 서비스 오류: {e}"
        except Exception as e:
            return f"오류 발생: {e}"
    
    def generate_factbomb_response(self, user_input):
        """팩폭 응답 생성"""
        # 키워드 기반 로컬 응답 (API 사용량 절약)
        for keyword, responses in self.factbomb_prompts.items():
            if keyword in user_input:
                return random.choice(responses)
        
        # 일반적인 팩폭 응답
        general_responses = [
            '현실을 직시하세요. 변명만 늘어놓고 있으면 아무것도 변하지 않아요!',
            '그런 마음가짐으로는 힘들 것 같은데요? 조금 더 적극적으로 접근해보세요!',
            '솔직히 말하면, 당신이 생각하는 것보다 상황이 간단해요. 행동만 하면 돼요!',
            '핑계 대신 계획을 세우세요. 그게 어른이 하는 일이에요!',
            '남 탓하지 말고 자기 탓부터 해보세요. 그래야 발전이 있어요!'
        ]
        
        return random.choice(general_responses)
    
    def generate_openai_response(self, user_input):
        """OpenAI API를 사용한 팩폭 응답 생성"""
        try:
            prompt = f"""
            당신은 친근하지만 현실적인 조언을 해주는 '팩폭 챗봇'입니다. 
            사용자의 질문에 대해 팩트를 기반으로 한 솔직한 답변을 해주세요.
            너무 상처를 주면서 까지 현실을 직시할 수 있도록 도와주세요.
            
            답변 스타일:
            - 친근하고 유머러스하게
            - 구체적인 해결방안 제시
            - 50자 내외로 간결하게
            - 한국어로 답변
            
            사용자 질문: {user_input}
            """
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0.8
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"OpenAI API 오류: {e}")
            return self.generate_factbomb_response(user_input)
    
    def text_to_speech_openai(self, text):
        """OpenAI TTS를 사용한 음성 생성"""
        try:
            with client.audio.speech.with_streaming_response.create(
                model='tts-1',
                voice='nova',
                input=text
            ) as response:
                # 메모리에서 직접 재생
                audio_data = response.content
                return Audio(audio_data, autoplay=True)
                
        except Exception as e:
            print(f"OpenAI TTS 오류: {e}")
            return self.text_to_speech_gtts(text)
    
    def text_to_speech_gtts(self, text):
        """gTTS를 사용한 음성 생성"""
        try:
            tts = gTTS(text=text, lang='ko')
            
            # 메모리 버퍼에 저장
            fp = io.BytesIO()
            tts.write_to_fp(fp)
            fp.seek(0)
            
            return Audio(fp.read(), autoplay=True)
            
        except Exception as e:
            print(f"gTTS 오류: {e}")
            return None
    
    def display_chat_history(self):
        """카카오톡 스타일로 채팅 내역 표시"""
        html_content = """
        <div style='max-height: 400px; overflow-y: auto; border: 2px solid #ddd; border-radius: 10px; padding: 15px; background-color: #f8f9fa;'>
        """
        
        for msg in self.chat_history:
            if msg['sender'] == 'user':
                html_content += f"""
                <div style='text-align: right; margin-bottom: 10px;'>
                    <div style='display: inline-block; background-color: #007bff; color: white; padding: 8px 12px; border-radius: 15px; border-bottom-right-radius: 3px; max-width: 70%;'>
                        {msg['text']}
                    </div>
                    <div style='font-size: 10px; color: #666; margin-top: 2px;'>나</div>
                </div>
                """
            else:
                html_content += f"""
                <div style='text-align: left; margin-bottom: 10px;'>
                    <div style='display: inline-block; background-color: white; color: #333; padding: 8px 12px; border-radius: 15px; border-bottom-left-radius: 3px; border: 1px solid #ddd; max-width: 70%;'>
                        {msg['text']}
                    </div>
                    <div style='font-size: 10px; color: #666; margin-top: 2px;'>팩폭봇</div>
                </div>
                """
        
        html_content += "</div>"
        return HTML(html_content)
    
    def chat_round(self, use_openai=False):
        """한 번의 대화 라운드 실행"""
        print("\n" + "="*50)
        print("🎤 음성 입력을 시작합니다...")
        
        # 1. STT: 음성 → 텍스트
        user_text = self.speech_to_text()
        print(f"👤 사용자: {user_text}")
        
        if "종료" in user_text or "끝" in user_text:
            print("👋 채팅을 종료합니다!")
            return False
        
        # 채팅 기록에 추가
        self.chat_history.append({"sender": "user", "text": user_text})
        
        # 2. AI 응답 생성
        if use_openai and OPENAI_API_KEY:
            bot_response = self.generate_openai_response(user_text)
        else:
            bot_response = self.generate_factbomb_response(user_text)
        
        print(f"🤖 팩폭봇: {bot_response}")
        
        # 채팅 기록에 추가
        self.chat_history.append({"sender": "bot", "text": bot_response})
        
        # 3. TTS: 텍스트 → 음성
        print("🔊 음성으로 재생합니다...")
        if use_openai and OPENAI_API_KEY:
            audio_widget = self.text_to_speech_openai(bot_response)
        else:
            audio_widget = self.text_to_speech_gtts(bot_response)
        
        if audio_widget:
            display(audio_widget)
        
        # 4. 채팅 UI 업데이트
        clear_output(wait=True)
        chat_display = self.display_chat_history()
        display(chat_display)
        
        return True

# ===== 셀 4: 챗봇 인스턴스 생성 =====

# 챗봇 초기화
bot = FactBombChatbot()

# ===== 셀 5: 단일 대화 테스트 =====

# 한 번의 대화 테스트 (OpenAI API 사용)
bot.chat_round(use_openai=True)

# ===== 셀 6: 연속 대화 모드 =====

def start_continuous_chat(use_openai=False):
    """연속 대화 모드 시작"""
    bot = FactBombChatbot()
    
    print("🤖 팩폭해주는 보이스 챗봇에 오신 것을 환영합니다!")
    print("💡 사용법:")
    print("   - 마이크에 대고 말씀하세요")
    print("   - '종료' 또는 '끝'이라고 말하면 종료됩니다")
    print("   - 'use_openai=True'로 설정하면 더 똑똑한 팩폭을 받을 수 있어요!")
    print("\n" + "="*50)
    
    while True:
        try:
            should_continue = bot.chat_round(use_openai=use_openai)
            if not should_continue:
                break
            
            print("\n⏰ 3초 후 다음 입력을 받습니다...")
            time.sleep(3)
            
        except KeyboardInterrupt:
            print("\n👋 사용자가 채팅을 종료했습니다!")
            break
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            print("🔄 다시 시도해주세요...")

# 연속 대화 시작 (로컬 모드)
# start_continuous_chat(use_openai=False)

# ===== 셀 7: 텍스트 입력 모드 (음성 없이 테스트) =====

def text_only_chat():
    """텍스트만으로 챗봇 테스트"""
    bot = FactBombChatbot()
    
    print("💬 텍스트 모드 팩폭 챗봇")
    print("'종료'를 입력하면 끝납니다.\n")
    
    while True:
        user_input = input("👤 당신: ")
        
        if user_input in ['종료', '끝', 'quit', 'exit']:
            print("👋 채팅 종료!")
            break
        
        # 팩폭 응답 생성
        bot_response = bot.generate_factbomb_response(user_input)
        print(f"🤖 팩폭봇: {bot_response}")
        
        # 채팅 기록 저장
        bot.chat_history.append({"sender": "user", "text": user_input})
        bot.chat_history.append({"sender": "bot", "text": bot_response})
        
        # TTS로 음성 재생
        try:
            audio_widget = bot.text_to_speech_gtts(bot_response)
            if audio_widget:
                display(audio_widget)
        except:
            print("🔇 음성 재생을 건너뜁니다...")
        
        print("-" * 30)

# 텍스트 모드 실행
text_only_chat()

# ===== 셀 8: 고급 OpenAI 통합 버전 =====

def create_advanced_factbomb_prompt(user_input, chat_history=[]):
    """고급 팩폭 프롬프트 엔지니어링"""
    
    # 이전 대화 컨텍스트 추가
    context = ""
    if chat_history:
        recent_history = chat_history[-4:]  # 최근 4개 메시지만
        for msg in recent_history:
            context += f"{msg['sender']}: {msg['text']}\n"
    
    prompt = f"""
당신은 '팩폭해주는 챗봇'입니다. 다음 규칙을 따라 답변해주세요:

**답변 스타일:**
- 친근하지만 현실적인 톤
- 팩트 기반의 솔직한 조언
- 유머와 위트가 섞인 표현
- 너무 상처주지 않되, 현실을 직시하게 도움
- 구체적인 해결방안이나 행동 제시
- 50-80자 내외로 적당한 길이

**금지사항:**
- 모욕적이거나 인격 모독적 표현
- 절망적이거나 부정적인 메시지
- 의학적/법적 조언

**이전 대화:**
{context}

**현재 사용자 입력:** {user_input}

**팩폭봇 답변:**
"""
    
    return prompt

def advanced_openai_response(user_input, chat_history=[]):
    """고급 OpenAI 팩폭 응답"""
    try:
        prompt = create_advanced_factbomb_prompt(user_input, chat_history)
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 더 저렴한 모델 사용
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150,
            temperature=0.8,
            presence_penalty=0.6,
            frequency_penalty=0.3
        )
        
        return response.choices[0].message.content.strip()
        
    except Exception as e:
        print(f"OpenAI API 오류: {e}")
        return bot.generate_factbomb_response(user_input)

# ===== 셀 9: 완전한 보이스 챗봇 실행 =====

def run_complete_voice_chatbot():
    """완전한 보이스 챗봇 실행"""
    bot = FactBombChatbot()
    
    print("🎯 완전한 팩폭 보이스 챗봇이 시작됩니다!")
    print("🔊 마이크 권한을 허용해주세요")
    print("💡 '종료'라고 말하면 끝납니다\n")
    
    # 웰컴 메시지
    welcome_msg = "안녕! 팩폭 전문 챗봇이야. 어떤 것에 대한 현실적인 조언이 필요하니?"
    print(f"🤖 팩폭봇: {welcome_msg}")
    
    # 웰컴 메시지 음성 재생
    welcome_audio = bot.text_to_speech_gtts(welcome_msg)
    if welcome_audio:
        display(welcome_audio)
    
    while True:
        try:
            print("\n" + "="*50)
            
            # 1. STT: 음성 입력
            user_text = bot.speech_to_text()
            
            if "음성을 인식할 수 없습니다" in user_text:
                print("🔄 다시 말씀해주세요...")
                continue
                
            print(f"👤 사용자: {user_text}")
            
            # 종료 조건
            if any(word in user_text for word in ['종료', '끝', '그만해줘']):
                final_msg = "응, 현실적인 조언이 도움이 되었길 바란다! 잘가라!"
                print(f"🤖 팩폭봇: {final_msg}")
                
                final_audio = bot.text_to_speech_gtts(final_msg)
                if final_audio:
                    display(final_audio)
                break
            
            # 채팅 기록 추가
            bot.chat_history.append({"sender": "user", "text": user_text})
            
            # 2. AI 응답 생성 (OpenAI 또는 로컬)
            if OPENAI_API_KEY:
                bot_response = advanced_openai_response(user_text, bot.chat_history)
            else:
                bot_response = bot.generate_factbomb_response(user_text)
            
            print(f"🤖 팩폭봇: {bot_response}")
            bot.chat_history.append({"sender": "bot", "text": bot_response})
            
            # 3. TTS: 음성 재생
            if OPENAI_API_KEY:
                audio_widget = bot.text_to_speech_openai(bot_response)
            else:
                audio_widget = bot.text_to_speech_gtts(bot_response)
            
            if audio_widget:
                display(audio_widget)
            
            # 4. 채팅 UI 업데이트
            chat_display = bot.display_chat_history()
            display(chat_display)
            
            print("\n⏰ 잠시 후 다음 입력을 받습니다...")
            time.sleep(2)
            
        except KeyboardInterrupt:
            print("\n👋 사용자가 채팅을 종료했습니다!")
            break
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            print("🔄 다시 시도합니다...")

# 완전한 보이스 챗봇 실행
# run_complete_voice_chatbot()

# ===== 셀 10: 개별 기능 테스트 =====

def test_individual_functions():
    """개별 기능들을 따로 테스트"""
    bot = FactBombChatbot()
    
    print("🧪 개별 기능 테스트\n")
    
    # 1. 팩폭 응답 테스트
    test_inputs = ["사랑 고민이 있어", "다이어트 하고 싶어", "돈을 벌고 싶어", "공부가 안 돼"]
    
    print("📝 팩폭 응답 테스트:")
    for test_input in test_inputs:
        response = bot.generate_factbomb_response(test_input)
        print(f"   Q: {test_input}")
        print(f"   A: {response}\n")
    
    # 2. TTS 테스트
    print("🔊 TTS 테스트:")
    test_text = "안녕하세요! 팩폭 챗봇입니다!"
    audio_widget = bot.text_to_speech_gtts(test_text)
    if audio_widget:
        display(audio_widget)
    
    # 3. 채팅 UI 테스트
    print("💬 채팅 UI 테스트:")
    bot.chat_history = [
        {"sender": "user", "text": "사랑 고민이 있어요"},
        {"sender": "bot", "text": "사랑은 호르몬의 장난이에요. 그래도 즐기세요!"},
        {"sender": "user", "text": "다이어트 하고 싶어요"},
        {"sender": "bot", "text": "칼로리 IN < 칼로리 OUT! 이것만 기억하세요!"}
    ]
    
    chat_display = bot.display_chat_history()
    display(chat_display)

# 개별 기능 테스트 실행
test_individual_functions()

# ===== 셀 11: 설정 및 사용법 =====

print("""
🎯 팩폭 보이스 챗봇 사용법

📋 준비사항:
1. .env 파일에 OPENAI_API_KEY 설정 (선택사항)
2. 마이크 권한 허용
3. 조용한 환경에서 테스트

🚀 실행 방법:

1️⃣ 텍스트만 테스트:
   text_only_chat()

2️⃣ 완전한 보이스 챗봇:
   run_complete_voice_chatbot()

3️⃣ 개별 기능 테스트:
   test_individual_functions()

⚙️ 설정 옵션:
- use_openai=True: OpenAI API 사용 (더 똑똑한 팩폭)
- use_openai=False: 로컬 응답 사용 (API 키 불필요)

🎤 음성 명령:
- "종료", "끝", "그만해줘" → 채팅 종료
- 일반 대화 → 팩폭 응답

💡 팩폭 키워드:
사랑, 다이어트, 성공, 돈, 공부 등의 키워드를 포함하면
더 구체적인 팩폭을 받을 수 있어요!
""")

# ===== 셀 12: 최종 실행 (이 셀을 실행하세요!) =====

# 최종 실행 - 원하는 모드를 선택하세요

print("어떤 모드로 실행하시겠습니까?")
print("1. 텍스트만 (text_only_chat)")
print("2. 완전한 보이스 챗봇 - 로컬 모드")  
print("3. 완전한 보이스 챗봇 - OpenAI 모드")

mode = input("모드를 선택하세요 (1, 2, 3): ")

if mode == "1":
    text_only_chat()
elif mode == "2":
    run_complete_voice_chatbot()
elif mode == "3":
    if OPENAI_API_KEY:
        print("OpenAI API 키가 설정되었습니다! 🚀")
        # OpenAI 모드로 실행
        def run_openai_voice_chatbot():
            bot = FactBombChatbot()
            while True:
                try:
                    should_continue = bot.chat_round(use_openai=True)
                    if not should_continue:
                        break
                    time.sleep(2)
                except KeyboardInterrupt:
                    break
        
        run_openai_voice_chatbot()
    else:
        print("❌ OpenAI API 키가 설정되지 않았습니다!")
        print("📝 .env 파일에 OPENAI_API_KEY를 설정해주세요")
        print("🔄 로컬 모드로 실행합니다...")
        run_complete_voice_chatbot()
else:
    print("기본 텍스트 모드로 실행합니다...")
    text_only_chat()